In [1]:
import os
import torch
from dotenv import load_dotenv
from huggingface_hub import login
from langchain_huggingface import HuggingFaceEndpoint, HuggingFaceEmbeddings, ChatHuggingFace
from langchain_chroma import Chroma
from langchain_community.document_loaders import PyPDFLoader, DirectoryLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
import gradio as gr

/Users/alial-taweel/projects/ai/rag_resumes_scanner/venv_rag_resumes_scanner/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# --- 1. SETUP & AUTH ---
load_dotenv()
hf_token = os.getenv("HUGGINGFACEHUB_API_TOKEN")
if hf_token:
    login(token=hf_token)
    print("✅ Authenticated")

✅ Authenticated


In [5]:
# --- 2. DEFINE THE MISSING FUNCTIONS (This fixes the NameError!) ---
def load_docs(directory):
    if not os.path.exists(directory):
        os.makedirs(directory)
        print(f"📁 Created directory '{directory}'. Add your PDFs there!")
    loader = DirectoryLoader(directory, glob="./*.pdf", loader_cls=PyPDFLoader)
    return loader.load()

def split_docs(documents):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
    return text_splitter.split_documents(documents)

def create_vector_store(chunks):
    # Embeddings stay local (they are small); only the LLM is cloud-based
    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    return Chroma.from_documents(
        documents=chunks, 
        embedding=embeddings,
        persist_directory="./db_resume",
        collection_name="resumes_collection"
    )

In [6]:

# --- 3. LLM CONFIGURATION (Cloud Version) ---
model_id = "meta-llama/Llama-3.2-3B-Instruct"

print(f"🌐 Connecting to Hugging Face API for {model_id}...")

# This does NOT download the model; it just creates a connection
raw_llm = HuggingFaceEndpoint(
    repo_id=model_id,
    task="text-generation",
    max_new_tokens=512,
    temperature=0.1,
    huggingfacehub_api_token=os.getenv("HUGGINGFACEHUB_API_TOKEN"),
)
# This wrapper solves the "conversational" task error
llm = ChatHuggingFace(llm=raw_llm)
print("✅ Cloud LLM Connection Ready!")

🌐 Connecting to Hugging Face API for meta-llama/Llama-3.2-3B-Instruct...
✅ Cloud LLM Connection Ready!
